In [133]:

import requests
from sqlalchemy import create_engine
import time
from datetime import datetime
import pandas as pd

In [134]:
#I am using the 1forge api for forex by fetching the most recent information about the Euro and US dollar trades
#The API gives information about the bid and ask amounts as well as the price of the latest trade executed
#I then store that data in my database called forex in the table called eur_to_usd

get_url = 'https://forex.1forge.com/1.0.3/quotes?pairs=EURUSD&api_key=qm6HG8iNbl9ElKzBfO3ClG2KqktwJ74q'
resp = requests.get(get_url)
data = resp.json()

In [135]:
#connect to database
conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = '35.196.137.214', 
    user = 'root',
    password = '8g4Cb5cgKdDOg4p7')

engine = create_engine(conn_string)
con = engine.connect()

In [136]:
# create a new schema
db_name = 'forex'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)
engine.execute(create_db_query)


/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1007, "Can't create database 'forex'; database exists")
  cursor.execute(statement, parameters)


In [137]:
# create a new table with time varient data in the database
table_name = 'eur_to_usd'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (ask float, 
                                bid float,
                                price float,
                                symbol varchar(10),
                                date_time datetime,
                                PRIMARY KEY(price, date_time)
                                )'''.format(db=db_name, table=table_name)
engine.execute(create_table_query)

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1050, "Table 'eur_to_usd' already exists")
  cursor.execute(statement, parameters)


In [138]:
#populate the table


query_template = '''INSERT IGNORE INTO {db}.{table}(ask,
                                                    bid,
                                                    price,
                                                    symbol,
                                                    date_time)
                                            VALUES (%s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
for entry in data:
    ask = float(entry['ask'])
    bid = float(entry['bid'])
    price = float(entry['price'])
    symbol = entry['symbol']
    datetime = datetime.fromtimestamp(entry['timestamp'])
    
    query_parameters = (ask, bid, price, symbol, datetime)
    try:
        engine.execute(query_template, query_parameters)
    except:
        break

In [139]:

check = pd.read_sql("SELECT * FROM forex.eur_to_usd", con=engine)
check

,ask,bid,price,symbol,date_time
0,1.15776,1.15775,1.15775,EURUSD,2018-10-03 04:10:47
1,1.15778,1.15777,1.15777,EURUSD,2018-10-03 04:21:56
2,1.15779,1.15778,1.15779,EURUSD,2018-10-03 04:13:09
3,1.15789,1.15788,1.15789,EURUSD,2018-10-03 04:22:29
